# Manipulation des données de nuscenes avec panda

In [90]:
%matplotlib inline

from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [91]:
# data/sets/nuscenes
# D:\Utilisateurs\Alexandre\Repertoire_D\nuscenes\v1 0-trainval01

#nusc = NuScenes(version='v1.0-mini', dataroot='../data/sets/nuscenes')
nusc = NuScenes(version='v1.0-trainval', dataroot='D:/Utilisateurs/Alexandre/Repertoire_D/nuscenes/v1.0-trainval01')

nusc_can = NuScenesCanBus(dataroot='../data/sets/nuscenes')



Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 49.1 seconds.
Reverse indexing ...
Done reverse indexing in 8.3 seconds.


In [221]:
#nusc.list_scenes()

In [222]:
scene_name = 'scene-0061'
dic_scene = nusc_can.get_messages(scene_name,'vehicle_monitor')
#print(test)
features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
df_scene = pd.DataFrame.from_dict(dic_scene)[features]
print(df_scene)

    vehicle_speed  steering  throttle  left_signal  right_signal
0           31.44       3.0         0            0             0
1           30.73       3.5         0            1             0
2           29.45       3.6         0            1             0
3           28.09       3.6         0            0             0
4           27.07       3.5         0            0             0
5           26.25       3.6         0            1             0
6           25.35       4.0         0            0             0
7           24.51       4.7         0            0             0
8           23.58       6.5         0            1             0
9           22.69      12.6         0            1             0
10          21.89      23.8         0            0             0
11          21.20      34.4         0            0             0
12          20.25      45.8         0            1             0
13          19.34      69.8         0            1             0
14          18.33      87

In [4]:
scene_name = 'scene-1100'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
# nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

## Premier test: essayer d'apprendre quand mettre le clignotant 
On va faire un essai pour prendre en main le dataset, avec seulement la vitesse, l'accélération et l'inclinaison du volant. Je ne pense pas qu'il soit possible de prévoir quand mettre un clignotant, il manque certaines informations (trajectoire notamment)

### 1/ Prétraitement des données

In [208]:
# Pour utiliser seulement nusc_can 
# (evite de charger nusc scene qui est plus lourd et ne sert actuellement qu'à prendre la liste des scenes)
# A faire 
def list_scene():
    pass

In [209]:
all_scene = [ s["name"] for s in nusc.scene ]
#print(all_scene)

In [210]:
# Mets des 1 tout le temps pour le clignotant au lieu d'une alternance par défaut
def fill_signal(df,signal):
    i = 0
    index = df.columns.get_loc(signal)
    while df[signal][i] != 1:
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        df.iat[i,index] = 1
        i += 1
    return df

In [211]:
#Ajoute une nouvelle colonne où les valeurs sont: 0(rien), 1(clignotant gauche), 2(clignotant droit)
#Plus pratique que "fill_signal" car on aura une seul colonne Y pour l'apprentissage
def add_signal_column(df,signal):
    i = 0
    tab = []
    while df[signal][i] != 1:
        tab += [0]
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        if signal == "right_signal":
            tab += [2]
        else:
            tab += [1]
        i += 1
    while i < len(df):
        tab += [0]
        i += 1
    df["signal"] = tab
    return df

In [212]:
blackint = nusc_can.can_blacklist
blacklist = [ "scene-0"+ str(i) for i in blackint]
print(blacklist)
print( "%s" in all_scene)

['scene-0161', 'scene-0162', 'scene-0163', 'scene-0164', 'scene-0165', 'scene-0166', 'scene-0167', 'scene-0168', 'scene-0170', 'scene-0171', 'scene-0172', 'scene-0173', 'scene-0174', 'scene-0175', 'scene-0176', 'scene-0309', 'scene-0310', 'scene-0311', 'scene-0312', 'scene-0313', 'scene-0314']
False


In [213]:
tab = []
for s in all_scene:
    if s not in blacklist and s not in ["scene-0419","scene-0420"] :
        dic_scene = nusc_can.get_messages(s,'vehicle_monitor')
        features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
        df_scene = pd.DataFrame.from_dict(dic_scene)[features]
        if df_scene["left_signal"].any():
            #df_scene = fill_signal(df_scene,"left_signal")
            new_df = add_signal_column(df_scene,"left_signal")
            #print(new_df)
        if df_scene["right_signal"].any():
            #df_scene = fill_signal(df_scene,"right_signal")
            new_df = add_signal_column(df_scene,"right_signal")
            #print(new_df)
        tab += [new_df]
    
df_total = pd.concat(tab)
print(len(all_scene))
print(df_total)

850
    vehicle_speed  steering  throttle  left_signal  right_signal  signal
0           14.74     191.9        55            0             0       0
1           14.63     206.4       125            0             0       0
2           15.19     206.3       158            1             0       0
3           16.10     192.3       182            1             0       0
4           17.24     120.1       202            0             0       0
..            ...       ...       ...          ...           ...     ...
35          39.21       2.1         0            0             1       2
36          36.06       3.1         0            0             0       2
37          32.50       2.4         0            0             0       2
38          29.00       1.3         0            0             1       2
39          25.50      -5.3         0            0             1       2

[33123 rows x 6 columns]


In [214]:
df_total.corr()

,vehicle_speed,steering,throttle,left_signal,right_signal,signal
vehicle_speed,1.000000,0.036777,0.240233,-0.161973,-0.189532,-0.399030
steering,0.036777,1.000000,-0.016328,0.251038,-0.235810,-0.228922
throttle,0.240233,-0.016328,1.000000,-0.083612,-0.085723,-0.181831
left_signal,-0.161973,0.251038,-0.083612,1.000000,-0.097808,0.041057
right_signal,-0.189532,-0.235810,-0.085723,-0.097808,1.000000,0.598528
signal,-0.399030,-0.228922,-0.181831,0.041057,0.598528,1.000000


### 2/ Apprentissage

In [215]:
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
from sklearn.ensemble import RandomForestClassifier
import random

In [216]:
features = ["vehicle_speed","steering","throttle"]
X = df_total[features]
y = df_total["signal"]
print(len(X)/20)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

1656.15


#### 2/1/ K plus proches voisins et SVM sur une seule variable signal a 3 valeurs

In [217]:
model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
print(model.score(X_test,y_test))

model2 = RandomForestClassifier(n_estimators=100,random_state = 42)
model2.fit(X_train,y_train)
model2.score(X_test,y_test)

0.7969811320754717


0.872754716981132

0.80 avec les 1000 scènes (can bus), c'est pas mal pour un modèle basique, on peut améliorer ça.
Mais le score reflète-t-il la vérité? Peut-être que dans la majorité de scènes dans le jeu de test le clignotant n'est pas utilisé et donc le score peut-être boosté? Vérifions ça. 

#### 2/2 Vérification du modèle

Matrice de confusion:

In [220]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,model.predict(X_test)))
print(confusion_matrix(y_test,model2.predict(X_test)))


[[3402  210  338]
 [ 288  549   63]
 [ 385   61 1329]]
[[3626  145  179]
 [ 169  688   43]
 [ 257   50 1468]]


Résultat de la matrice de confusion sur le jeu de test (170 scènes, 6625 échantillons) avec k voisins (gauche) et une forêt aléatoire (droite):
- [3402,  210,  338]-----[3626  145  179]
- [ 288,  549,   63]--------[169  688   43] 
- [ 385,   61, 1329]------[ 257   50 1468]

K voisins:
Résultats assez mitigés le taux de d'erreur pour le clignotant gauche est de 0.5 et du clignotant droit 0.25.

Forêt aléatoire:
Résultats plutôt bon, taux d'erreur pour le clignotant gauche est de 0.30 et du clignotant droit 0.15.

J'obtiens des résultats quasi-identique en changeant l'attribution des valeurs dans la colonne signal (clignotant gauche = 2 au lieu de 1). 
On peut par ailleurs noter qu'il y a un double des cas où le clignotant droite est activé par rapport au clignotant gauche, cela peut être l'une des raisons qui explique la différence de score.

## Deuxième test: anticiper la vitesse avec un véhicule en face

Principe de base (idée): 
    - le système de pilotage envoie les postions gps à suivre, la vitesse et l'angle recommandés à l'algorithme de gestion
    - l'algorithme de gestion en fonction de la situation va envoyer sur un modèle pour donner une nouvelle valeur de la vitesse et de l'angle par rapport à l'environnement
    - plusieurs modèles, mais en premier un basique : s'il y a personne en face on change rien, sinon calcul de la nouvelle vitesse par rapport au véhicule devant.

Pour cela, il nous faut plusieurs données pour ce modèle:
    - position, vitesse, orientation du véhicule égo (nous) et du véhicule en face
    - à compléter?



### 1/ Recherche de ces données:

In [14]:
scene_test = nusc.scene[0]
scene_test

{'token': 'cc8c0bf57f984915a77078b10eb33198',
 'log_token': '7e25a2c8ea1f41c5b0da1e69ecfa71a2',
 'nbr_samples': 39,
 'first_sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'last_sample_token': 'ed5fc18c31904f96a8f0dbb99ff069c0',
 'name': 'scene-0061',
 'description': 'Parked truck, construction, intersection, turn left, following a van'}

In [15]:
sample = nusc.get('sample',scene_test['first_sample_token'])
ann = sample['anns'][18]
ann_meta = nusc.get('sample_annotation', ann)
ann_meta

{'token': '83d881a6b3d94ef3a3bc3b585cc514f8',
 'sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'instance_token': 'e91afa15647c4c4994f19aeb302c7179',
 'visibility_token': '4',
 'attribute_tokens': ['58aa28b1c2a54dc88e169808c07331e3'],
 'translation': [409.989, 1164.099, 1.623],
 'size': [2.877, 10.201, 3.595],
 'rotation': [-0.5828819500503033, 0.0, 0.0, 0.812556848660791],
 'prev': '',
 'next': 'f3721bdfd7ee4fd2a4f94874286df471',
 'num_lidar_pts': 495,
 'num_radar_pts': 13,
 'category_name': 'vehicle.truck'}